In [1]:
from ultralytics import YOLO
model=YOLO("yolov8m.pt")

In [3]:
model.train(data="/mnt/c/Users/Rahul/Desktop/Document-and-Record-Management/YOLOv8/data/data.yaml",epochs=70)

Ultralytics YOLOv8.2.77 🚀 Python-3.10.14 torch-2.4.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/mnt/c/Users/Rahul/Desktop/Document-and-Record-Management/YOLOv8/data/data.yaml, epochs=70, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fa

100%|██████████| 755k/755k [00:00<00:00, 2.40MB/s]

Overriding model.yaml nc=80 with nc=11

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           


  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384, 576, 3, 2]              
  8                  -1  2   3985920  ultralytics.nn.modules.block.C2f             [576, 576, 2, True]           
  9                  -1  1    831168  ultralytics.nn.modules.block.SPPF            [576, 576, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  2   1993728  ultralytics.nn.modules.block.C2f             [960, 384, 2]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None

100%|██████████| 6.25M/6.25M [00:00<00:00, 12.5MB/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       9/70      7.12G      0.517     0.3017     0.8526         44        640: 100%|██████████| 226/226 [01:33<00:00,  2.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:04<00:00,  1.85it/s]


                   all        239       2629      0.997      0.985      0.995      0.907

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/70       7.2G     0.5034     0.2979     0.8479        283        640:  31%|███       | 69/226 [12:21<28:06, 10.74s/it]


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [4]:
yolo = YOLO("/mnt/c/Users/Rahul/Desktop/Document-and-Record-Management/YOLOv8/runs/detect/train/weights/best.pt"); yolo.predict(source="/mnt/c/Users/Rahul/Desktop/Document-and-Record-Management/YOLOv8/pratikshya.jpg", save=True)



image 1/1 /mnt/c/Users/Rahul/Desktop/Document-and-Record-Management/YOLOv8/pratikshya.jpg: 480x640 1 citizenship_number, 67.9ms
Speed: 3.3ms preprocess, 67.9ms inference, 62.9ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'citizenship_number', 1: 'day', 2: 'district', 3: 'document_type', 4: 'father_name', 5: 'gender', 6: 'month', 7: 'municipality', 8: 'name', 9: 'ward', 10: 'year'}
 obb: None
 orig_img: array([[[ 79,  97, 104],
         [ 75,  93, 100],
         [ 71,  89,  96],
         ...,
         [ 81,  90, 104],
         [ 81,  86, 101],
         [ 78,  83,  98]],
 
        [[ 77,  95, 102],
         [ 76,  94, 101],
         [ 76,  94, 101],
         ...,
         [ 77,  86, 100],
         [ 79,  84,  99],
         [ 78,  83,  98]],
 
        [[ 91, 109, 116],
         [ 99, 117, 124],
         [111, 129, 136],
         ...,
         [ 75,  84,  98],
         [ 81,  86, 101],
         [ 83,  88, 103]],
 
        ...,
 
        [[ 34,  45,  65],
         [ 35,  46,  66],
         [ 37,  48,  68],
         ...,
         [ 95, 107, 119],
         [ 9

In [2]:
model = YOLO("")
results = model('/mnt/c/Users/Rahul/Desktop/Document-and-Record-Management/YOLOv8/Susmita Malla front.jpg')

# Extract the coordinates, class IDs, and confidence scores
for result in results:
    boxes = result.boxes.xyxy  # x1, y1, x2, y2 (top-left and bottom-right corner coordinates)
    scores = result.boxes.conf  # Confidence scores
    class_ids = result.boxes.cls  # Class IDs

    for box, score, class_id in zip(boxes, scores, class_ids):
        x1, y1, x2, y2 = box.tolist()  # Convert to list to get the values
        print(f"Class ID: {class_id}, Confidence: {score}, Coordinates: ({x1}, {y1}), ({x2}, {y2})")


image 1/1 /mnt/c/Users/Rahul/Desktop/Document-and-Record-Management/YOLOv8/Susmita Malla front.jpg: 512x640 (no detections), 56.3ms
Speed: 4.9ms preprocess, 56.3ms inference, 43.2ms postprocess per image at shape (1, 3, 512, 640)


In [ ]:
import cv2
from ultralytics import YOLO
from paddleocr import PaddleOCR
import os
import numpy as np

# Define the paths
image_path = '/mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/YOLOv8/mainali.jpg'
model_path = "/mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/pretrained models/best.pt"

# Load the trained YOLO model
model = YOLO(model_path)

# Perform inference on an image
results = model(image_path)

# Load the image with OpenCV
image = cv2.imread(image_path)

# Initialize PaddleOCR reader (English language model)
ocr = PaddleOCR(use_angle_cls=True, lang='en') 

# Dictionary to map class IDs to class names
class_map = {
    0: 'address',
    1: 'blood_group',
    2: 'citizenship_number',
    3: 'contact_number',
    4: 'dob',
    5: 'document_type',
    6: 'father_name',
    7: 'license_category',
    8: 'license_number',
    9: 'name'
}

# Generate a color map for classes
def get_color(class_id):
    np.random.seed(class_id)  # Seed for reproducibility
    return tuple(np.random.randint(0, 255, 3).tolist())

# Extract the base name of the image file (without extension)
base_name = os.path.splitext(os.path.basename(image_path))[0]

# Directory to save OCR results
output_dir = 'OCR_Outputs'
os.makedirs(output_dir, exist_ok=True)

# File to save OCR results with the same name as the image
output_file_path = os.path.join(output_dir, f'{base_name}_results.txt')

# Open the file in write mode
with open(output_file_path, 'w') as file:
    # Iterate over each result
    for result in results:
        boxes = result.boxes.xyxy  # Bounding box coordinates
        class_ids = result.boxes.cls  # Class IDs

        # Iterate over each detected box and class ID
        for box, class_id in zip(boxes, class_ids):
            x1, y1, x2, y2 = map(int, box.tolist())  # Convert coordinates to integers
            
            # Crop the image based on the coordinates
            cropped_img = image[y1:y2, x1:x2]

            # Use PaddleOCR to read the text from the cropped image
            ocr_result = ocr.ocr(cropped_img)

            # Extract and save the text recognized by PaddleOCR
            for line in ocr_result:
                for text in line:
                    file.write(f"Class: {class_map.get(int(class_id), 'Unknown')}, Text: {text[1][0]}\n")
                    
            # Draw bounding box and class label on the original image
            color = get_color(int(class_id))
            label = class_map.get(int(class_id), 'Unknown')
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# Save the annotated image to the same folder
annotated_image_path = os.path.join(output_dir, f'{base_name}_annotated.jpg')
cv2.imwrite(annotated_image_path, image)
